In [1]:
#%pip install emoji

import pandas as pd
import nltk
from nltk.corpus import stopwords
import emoji
import re

In [2]:
# # Download the stopwords collection from library
nltk.download('stopwords')
# Put it into a set to guarantee each word only appear once
STOPWORDS = list(set(stopwords.words('english')))
# Add punctuation to the stopwords list
STOPWORDS += [".", "!", "?", ",", ";", ":", "[", "]", "{", "}", "-", "+", 
    "_", "/", "#", "@", "$", "%", "^", "&", "*", "(", ")", "<", ">", "|", "=",
    ".-", ".,", "'", '"', ',"', ".>", ".<"]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kingu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("data_sample_nlp.csv")
df_en = df[df.language=='en'].copy()

C:\Users\kingu\AppData\Local\Temp\ipykernel_12104\3356171025.py:1: DtypeWarning: Columns (2,7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data_sample_nlp.csv")


In [3]:
df_en

label                    id           author_id  \
1       human  t1502924023386152961            44832407   
3       human  t1498816627898527744            80693776   
4       human  t1372582431639896064           422982642   
5       human  t1494068483881148419           121391116   
8       human  t1498386351883886594            29858496   
...       ...                   ...                 ...   
299998  human  t1288609934809149440            96768148   
299999  human  t1471504881882075140            17268418   
300000  human  t1499064851347304449            33152005   
300001  human  t1492208040795324425            76086369   
300002  human   t942173606888595457  709501413819293696   

                       created_at       type  \
1       2022-03-13 08:26:24+00:00  retweeted   
3       2022-03-02 00:25:05+00:00       post   
4       2021-03-18 16:15:27+00:00       post   
5       2022-02-16 21:57:39+00:00       post   
8       2022-02-28 19:55:19+00:00       post   
...                           ...        ...   
299998  2020-07-29 22:58:44+00:00       post   
299999  2021-12-16 15:37:57+00:00       post   
300000  2022-03-02 16:51:26+00:00       post   
300001  2022-02-11 18:44:55+00:00       post   
300002  2017-12-16 23:24:27+00:00       post   

                                                     text language  \
1       RT @Wieneraaron: When you need your emotional ...       en   
3       RT @InfoSecSherpa: "The Thurgood Marshall Coll...       en   
4       RT @NCIAorg: BREAKING – SAFE Banking Act Reint...       en   
5              @L1nds Oooh I like https://t.co/Y5V50GCHdB       en   
8             No war!.....PLEASE! https://t.co/YGxOZpN5Kk       en   
...                                                   ...      ...   
299998  @ldpsci I mean, you can *submit* it... :) But ...       en   
299999  RT @ElemPE1: "Toys for Tots"🎁\n\nLess lecturin...       en   
300000                                  @DavidVimes Same.       en   
300001          @BigelowLab I spy a @wellsreserve alumna!       en   
300002  RT @MPSHaringey: Who thinks #PDBoots deserves ...       en   

       geo_location referenced_tweets_types  referenced_tweets_ids  \
1             False             {retweeted}  {1502647106753875973}   
3             False                      {}                     {}   
4             False                      {}                     {}   
5             False                      {}                     {}   
8             False                      {}                     {}   
...             ...                     ...                    ...   
299998        False                      {}                     {}   
299999        False                      {}                     {}   
300000        False                      {}                     {}   
300001        False                      {}                     {}   
300002        False                      {}                     {}   

               user_mentions media_types  in_reply_to_user_id  \
1               {1243057506}          {}                  NaN   
3               {3208239154}          {}                  NaN   
4                {217618731}          {}                  NaN   
5                 {23075999}     {photo}         2.307600e+07   
8                         {}     {photo}                  NaN   
...                      ...         ...                  ...   
299998           {113481149}          {}         1.134811e+08   
299999  {907432611319914497}          {}                  NaN   
300000  {966462024145391616}          {}         9.664620e+17   
300001  {260908660,76086369}          {}         2.609087e+08   
300002           {407149465}          {}                  NaN   

       possibly_sensitive  
1                   False  
3                   False  
4                   False  
5                   False  
8                   False  
...                   ...  
299998              False  
299999              False  
300000 

In [3]:
def has_stopwords(df):
    return len(df[df['text'].apply(lambda x: len(set(x.split()) & set(STOPWORDS)))>0])

def has_links(df):
    return len(df[df['text'].str.contains('https')])

def has_tags(df):
    return len(df[df['text'].str.contains('@')])

def has_numbers(df):
    return len(df[df['text'].apply(lambda x: any(y.isnumeric() for y in x))])

def has_emojis(df):
    return len(df[df['text'].apply(lambda x: len(set(x.split()) & set(list(emoji.EMOJI_DATA.keys()))))>0])

def has_hashtag(df):
    return len(df[df['text'].str.contains('#')])

def has_rt(df):
    return len(df[df['text'].str.contains('RT')])


def analytics(df):
    human = df[df['label']=='human']
    bot = df[df['label']=='bot']
    
    stats = pd.DataFrame(index=['human','bot'], columns=['stopwords', 'links', 'tags', 'numbers', 'emojis', 'hashtag', 'rt'])
    
    stats.loc['human', 'stopwords'] = has_stopwords(human)
    stats.loc['bot', 'stopwords'] = has_stopwords(bot)
    
    stats.loc['human', 'links'] = has_links(human)
    stats.loc['bot', 'links'] = has_links(bot)
    
    stats.loc['human', 'tags'] = has_tags(human)
    stats.loc['bot', 'tags'] = has_tags(bot)
    
    stats.loc['human', 'numbers'] = has_numbers(human)
    stats.loc['bot', 'numbers'] = has_numbers(bot)
    
    stats.loc['human', 'emojis'] = has_emojis(human)
    stats.loc['bot', 'emojis'] = has_emojis(bot)
    
    stats.loc['human', 'hashtag'] = has_hashtag(human)
    stats.loc['bot', 'hashtag'] = has_hashtag(bot)
    
    stats.loc['human', 'rt'] = has_rt(human)
    stats.loc['bot', 'rt'] = has_rt(bot)    
    
    return stats    

In [94]:
df_stats = analytics(df_en)
df_stats

stopwords  links    tags numbers emojis hashtag     rt
human    183928  87080  142106  117902  21720   51534  77487
bot       15762   7917   12041   10823   1806    5995   6057

In [5]:
bots_df = pd.read_csv("bots_100k.csv")
stats_bots = analytics(bots_df)

In [6]:
stats_bots

stopwords  links   tags numbers emojis hashtag     rt
human         0      0      0       0      0       0      0
bot       90446  44155  68469   60850   9574   29749  36466

In [7]:
human_df = pd.read_csv("human_100k.csv")
stats_human = analytics(human_df)

In [8]:
stats_human

stopwords  links   tags numbers emojis hashtag     rt
human     90919  41358  70862   56836  11146   22277  34206
bot           0      0      0       0      0       0      0